In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [33]:
with np.load('/Users/KP/Desktop/UfT/ECE521_Winter_2018/Assignment 3/Assignment 3/notMNIST.npz') as data :
    Data, Target = data ["images"], data["labels"]
    np.random.seed(521)
    randIndx = np.arange(len(Data))
    np.random.shuffle(randIndx)
    Data = Data[randIndx]/255.
    Target = Target[randIndx]
    trainData, trainTarget = Data[:15000], Target[:15000]
    validData, validTarget = Data[15000:16000], Target[15000:16000]
    testData, testTarget = Data[16000:], Target[16000:]

In [34]:
trainZeros=np.zeros((15000, 10))
trainZeros[np.arange(15000),trainTarget]=1
trainTarget = trainZeros
validZeros=np.zeros((1000, 10))
validZeros[np.arange(1000),validTarget]=1
validTarget = validZeros
testZeros=np.zeros((2724, 10))
testZeros[np.arange(2724),testTarget]=1
testTarget = testZeros

In [83]:
#Hyperparameters to chnage

np.random.seed(seed=60749150)

learning_rate = np.asscalar(np.exp(np.random.uniform(low=-7.5, high=-4.5, size=1)))

num_layers = int(np.random.randint(1,6,1))

hidden_units = int(np.random.randint(100,500,1))

weight_decay = np.asscalar(np.exp(np.random.uniform(low=-7.5, high=-4.5, size=1)))

#0 no dropout, 1 dropout

dropout = np.random.randint(0,2,1)

print('Learning Rate:',learning_rate,'Number Layers:',num_layers,'Hidden Units:', hidden_units,'Weight Decay:',weight_decay,\
      'No Dropout',dropout)

n_epochs = 5000
batch_size = 500
n_dim = 28*28

Learning Rate: 0.0024606117272511774 Number Layers: 5 Hidden Units: 371 Weight Decay: 0.0036402635633598934 No Dropout [0]


In [73]:
def grab_batches(trainData, trainTarget, batch_size):
    batch_indices = np.random.permutation(range(15000)).reshape(-1, batch_size)
    X_batches = trainData.reshape(-1, n_dim)[batch_indices]
    y_batches = trainTarget[batch_indices]
    batches = zip(X_batches, y_batches)
    return batches

In [74]:
X = tf.placeholder(tf.float32,[None,n_dim])
Y = tf.placeholder(tf.float32,[None,10])

In [75]:
def hidden_layer(X, hidden_units):
    x_dimension = X.shape[1].value
    initializer = tf.contrib.layers.xavier_initializer(uniform=False)
    hidden_weights = tf.Variable(initializer([x_dimension, hidden_units]), name='weights')
    hidden_biases = tf.Variable(tf.zeros(hidden_units), name='biases')
    
    return tf.add(tf.matmul(X, hidden_weights), hidden_biases), hidden_weights

In [84]:
y_, W1 = hidden_layer(X,hidden_units)
y_, W2 = hidden_layer(X,hidden_units)
y_, W3 = hidden_layer(X,hidden_units)
y_, W4 = hidden_layer(X,hidden_units)
y_, W5 = hidden_layer(X,hidden_units)
y_, W6 = hidden_layer(tf.nn.relu(y_),10)

y_ = tf.nn.dropout(y_, keep_prob=0.5)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=y_))
regularizer = (tf.nn.l2_loss(W1) + tf.nn.l2_loss(W2) + tf.nn.l2_loss(W3) + tf.nn.l2_loss(W4)+tf.nn.l2_loss(W5)+tf.nn.l2_loss(W6)) 
loss = tf.reduce_mean(loss + weight_decay * regularizer)

prediction = tf.cast(tf.round(tf.argmax(y_,1)), tf.int8)
equality = tf.equal(prediction, tf.cast(tf.argmax(Y,1), tf.int8))
accuracy = tf.reduce_mean(tf.cast(equality, tf.float32))

training_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)
init = tf.global_variables_initializer()

In [85]:
valid_accuracies = []
train_accuracies = []
test_accuracies = []

In [86]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(1,n_epochs+1):
        batches = grab_batches(trainData, trainTarget, batch_size)
        for X_batch, y_batch in batches:
            sess.run(training_step, feed_dict={X: X_batch, Y: y_batch})
        feed_dict ={X: trainData.reshape(-1,n_dim), Y: trainTarget}
        train_loss, train_accuracy = sess.run([loss, accuracy], feed_dict)
        print("Epoch: {}, Loss: {}, Accuracy: {}".format(epoch, train_loss, train_accuracy))
    train_accuracies.append(train_accuracy)
    valid_accuracy = sess.run(accuracy, feed_dict = {X: validData.reshape(-1,n_dim), Y: validTarget})
    valid_accuracies.append(valid_accuracy)
    test_accuracy = sess.run(accuracy, feed_dict = {X: testData.reshape(-1,n_dim), Y: testTarget})
    test_accuracies.append(test_accuracy)

Epoch: 1, Loss: 1.2607650756835938, Accuracy: 0.887066662311554
Epoch: 2, Loss: 0.8986546397209167, Accuracy: 0.9050666689872742
Epoch: 3, Loss: 0.7247710227966309, Accuracy: 0.9113333225250244
Epoch: 4, Loss: 0.6050732135772705, Accuracy: 0.9188666939735413
Epoch: 5, Loss: 0.5294938683509827, Accuracy: 0.9221333265304565
Epoch: 6, Loss: 0.48273965716362, Accuracy: 0.9233333468437195
Epoch: 7, Loss: 0.4467528462409973, Accuracy: 0.9284666776657104
Epoch: 8, Loss: 0.42441481351852417, Accuracy: 0.9279333353042603
Epoch: 9, Loss: 0.39831244945526123, Accuracy: 0.9311333298683167
Epoch: 10, Loss: 0.3857584595680237, Accuracy: 0.9344666600227356
Epoch: 11, Loss: 0.3800400495529175, Accuracy: 0.9358000159263611
Epoch: 12, Loss: 0.370542049407959, Accuracy: 0.9366000294685364
Epoch: 13, Loss: 0.3635272979736328, Accuracy: 0.9390666484832764
Epoch: 14, Loss: 0.36811473965644836, Accuracy: 0.9384666681289673
Epoch: 15, Loss: 0.35428255796432495, Accuracy: 0.9415333271026611
Epoch: 16, Loss: 0.

In [87]:
valid_accuracies
test_accuracies

[0.92799997]

[0.91593248]

In [88]:
1-0.92799997
1-0.91593248

0.07200002999999999

0.08406751999999995